In [1]:
%matplotlib inline

import pandas as pd

import tensorflow as tf
tf.enable_eager_execution()

import numpy as np
import os
import time
tf.VERSION

'1.13.1'

In [2]:

gb_path = './goodbooks/'
t = pd.read_csv( gb_path + 'tags.csv' )
bt = pd.read_csv( gb_path + 'book_tags.csv')
b = pd.read_csv( gb_path + 'books.csv' )

In [3]:
bt.head()

,goodreads_book_id,tag_id,count
0,1,30574,167697
1,1,11305,37174
2,1,11557,34173
3,1,8717,12986
4,1,33114,12716


In [4]:
mystery_tags = t[t.tag_name.str.contains("myster")]
mystery_tags.head()

,tag_id,tag_name
412,412,1920-30-s-british-period-mystery
1000,1000,3-mystery-thrillers
1445,1445,abduction-lisa-gardner-mystery-susp
1476,1476,about-to-read-mystery-thriller
1548,1548,action-mystery


In [5]:
mystery_book_ids = bt[bt.tag_id.isin(mystery_tags.tag_id)].goodreads_book_id.unique()
mystery_books = b[b.goodreads_book_id.isin(mystery_book_ids)]
mystery_books.head(100)

,book_id,goodreads_book_id,best_book_id,work_id,books_count,isbn,isbn13,authors,original_publication_year,original_title,...,ratings_count,work_ratings_count,work_text_reviews_count,ratings_1,ratings_2,ratings_3,ratings_4,ratings_5,image_url,small_image_url
1,2,3,3,4640799,491,439554934,9.780440e+12,"J.K. Rowling, Mary GrandPré",1997.0,Harry Potter and the Philosopher's Stone,...,4602479,4800065,75867,75504,101676,455024,1156318,3011543,https://images.gr-assets.com/books/1474154022m...,https://images.gr-assets.com/books/1474154022s...
3,4,2657,2657,3275794,487,61120081,9.780061e+12,Harper Lee,1960.0,To Kill a Mockingbird,...,3198671,3340896,72586,60427,117415,446835,1001952,1714267,https://images.gr-assets.com/books/1361975680m...,https://images.gr-assets.com/books/1361975680s...
8,9,960,960,3338963,311,1416524797,9.781417e+12,Dan Brown,2000.0,Angels & Demons,...,2001311,2078754,25112,77841,145740,458429,716569,680175,https://images.gr-assets.com/books/1303390735m...,https://images.gr-assets.com/books/1303390735s...
15,16,2429135,2429135,1708725,274,307269752,9.780307e+12,"Stieg Larsson, Reg Keeland",2005.0,Män som hatar kvinnor,...,1808403,1929834,62543,54835,86051,285413,667485,836050,https://images.gr-assets.com/books/1327868566m...,https://images.gr-assets.com/books/1327868566s...
17,18,5,5,2402163,376,043965548X,9.780440e+12,"J.K. Rowling, Mary GrandPré, Rufus Beck",1999.0,Harry Potter and the Prisoner of Azkaban,...,1832823,1969375,36099,6716,20413,166129,509447,1266670,https://images.gr-assets.com/books/1499277281m...,https://images.gr-assets.com/books/1499277281s...
20,21,2,2,2809203,307,439358078,9.780439e+12,"J.K. Rowling, Mary GrandPré",2003.0,Harry Potter and the Order of the Phoenix,...,1735368,1840548,28685,9528,31577,180210,494427,1124806,https://images.gr-assets.com/books/1387141547m...,https://images.gr-assets.com/books/1387141547s...
21,22,12232938,12232938,1145090,183,316166685,9.780316e+12,Alice Sebold,2002.0,The Lovely Bones,...,1605173,1661562,36642,62777,131188,404699,583575,479323,https://images.gr-assets.com/books/1457810586m...,https://images.gr-assets.com/books/1457810586s...
22,23,15881,15881,6231171,398,439064864,9.780439e+12,"J.K. Rowling, Mary GrandPré",1998.0,Harry Potter and the Chamber of Secrets,...,1779331,1906199,34172,8253,42251,242345,548266,1065084,https://images.gr-assets.com/books/1474169725m...,https://images.gr-assets.com/books/1474169725s...
23,24,6,6,3046572,332,439139600,9.780439e+12,"J.K. Rowling, Mary GrandPré",2000.0,Harry Potter and the Goblet of Fire,...,1753043,1868642,31084,6676,20210,151785,494926,1195045,https://images.gr-assets.com/books/1361482611m...,https://images.gr-assets.com/books/1361482611s...
24,25,136251,136251,2963218,263,545010225,9.780545e+12,"J.K. Rowling, Mary GrandPré",2007.0,Harry Potter and the Deathly Hallows,...,1746574,1847395,51942,9363,22245,113646,383914,1318227,https://images.gr-assets.com/books/1474171184m...,https://images.gr-assets.com/books/1474171184s...


In [6]:
text  = '\n'.join([title for title in mystery_books.original_title.values if type(title)==str])
print(text)

Harry Potter and the Philosopher's Stone
To Kill a Mockingbird
Angels & Demons 
Män som hatar kvinnor
Harry Potter and the Prisoner of Azkaban
Harry Potter and the Order of the Phoenix
The Lovely Bones
Harry Potter and the Chamber of Secrets
Harry Potter and the Goblet of Fire
Harry Potter and the Deathly Hallows
The Da Vinci Code
Harry Potter and the Half-Blood Prince
Gone Girl
The Lightning Thief
Jane Eyre
City of Bones
The Curious Incident of the Dog in the Night-Time
The Girl on the Train
Northern Lights
Frankenstein; or, The Modern Prometheus
The Shining
Looking for Alaska
Holes
Jurassic Park
A Time to Kill
Paper Towns
The Maze Runner
The Secret Garden
The Picture of Dorian Gray
Dracula
Flickan som lekte med elden
Le Comte de Monte-Cristo
The Memory Keeper's Daughter
The Firm
Room
Miss Peregrine’s Home for Peculiar Children
Luftslottet som sprängdes
The Pillars of the Earth
Deception Point
The Goldfinch
The Battle of the Labyrinth
The Lost Hero
American Gods
The Stand
Digital Fort

In [7]:
# The unique characters in the file
vocab = sorted(set(text))
print ('{} unique characters'.format(len(vocab)))

325 unique characters


In [8]:
# Creating a mapping from unique characters to indices
char2idx = {u:i for i, u in enumerate(vocab)}
idx2char = np.array(vocab)

text_as_int = np.array([char2idx[c] for c in text])

In [9]:
print('{')
for char,_ in zip(char2idx, range(200)):
    print('  {:4s}: {:3d},'.format(repr(char), char2idx[char]))
print('  ...\n}')

{
  '\n':   0,
  ' ' :   1,
  '!' :   2,
  '"' :   3,
  '#' :   4,
  '&' :   5,
  "'" :   6,
  '(' :   7,
  ')' :   8,
  '+' :   9,
  ',' :  10,
  '-' :  11,
  '.' :  12,
  '/' :  13,
  '0' :  14,
  '1' :  15,
  '2' :  16,
  '3' :  17,
  '4' :  18,
  '5' :  19,
  '6' :  20,
  '7' :  21,
  '8' :  22,
  '9' :  23,
  ':' :  24,
  ';' :  25,
  '?' :  26,
  'A' :  27,
  'B' :  28,
  'C' :  29,
  'D' :  30,
  'E' :  31,
  'F' :  32,
  'G' :  33,
  'H' :  34,
  'I' :  35,
  'J' :  36,
  'K' :  37,
  'L' :  38,
  'M' :  39,
  'N' :  40,
  'O' :  41,
  'P' :  42,
  'Q' :  43,
  'R' :  44,
  'S' :  45,
  'T' :  46,
  'U' :  47,
  'V' :  48,
  'W' :  49,
  'X' :  50,
  'Y' :  51,
  'Z' :  52,
  '[' :  53,
  ']' :  54,
  'a' :  55,
  'b' :  56,
  'c' :  57,
  'd' :  58,
  'e' :  59,
  'f' :  60,
  'g' :  61,
  'h' :  62,
  'i' :  63,
  'j' :  64,
  'k' :  65,
  'l' :  66,
  'm' :  67,
  'n' :  68,
  'o' :  69,
  'p' :  70,
  'q' :  71,
  'r' :  72,
  's' :  73,
  't' :  74,
  'u' :  75,
  'v' :  7

In [10]:
# The maximum length sentence we want for a single input in characters
seq_length = 100
examples_per_epoch = len(text)//seq_length

# Create training examples / targets
char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)

for i in char_dataset.take(5):
  print(idx2char[i.numpy()])

Instructions for updating:
Colocations handled automatically by placer.
H
a
r
r
y


In [11]:
sequences = char_dataset.batch(seq_length+1, drop_remainder=True)

for item in sequences.take(5):
  print(repr(''.join(idx2char[item.numpy()])))

"Harry Potter and the Philosopher's Stone\nTo Kill a Mockingbird\nAngels & Demons \nMän som hatar kvinnor"
'\nHarry Potter and the Prisoner of Azkaban\nHarry Potter and the Order of the Phoenix\nThe Lovely Bones\n'
'Harry Potter and the Chamber of Secrets\nHarry Potter and the Goblet of Fire\nHarry Potter and the Deat'
'hly Hallows\nThe Da Vinci Code\nHarry Potter and the Half-Blood Prince\nGone Girl\nThe Lightning Thief\nJa'
'ne Eyre\nCity of Bones\nThe Curious Incident of the Dog in the Night-Time\nThe Girl on the Train\nNorther'


In [12]:
def split_input_target(chunk):
    input_text = chunk[:-1]
    target_text = chunk[1:]
    return input_text, target_text

dataset = sequences.map(split_input_target)

In [13]:
for input_example, target_example in  dataset.take(1):
  print ('Input data: ', repr(''.join(idx2char[input_example.numpy()])))
  print ('Target data:', repr(''.join(idx2char[target_example.numpy()])))

Input data:  "Harry Potter and the Philosopher's Stone\nTo Kill a Mockingbird\nAngels & Demons \nMän som hatar kvinno"
Target data: "arry Potter and the Philosopher's Stone\nTo Kill a Mockingbird\nAngels & Demons \nMän som hatar kvinnor"


In [14]:
for i, (input_idx, target_idx) in enumerate(zip(input_example[:5], target_example[:5])):
    print("Step {:4d}".format(i))
    print("  input: {} ({:s})".format(input_idx, repr(idx2char[input_idx])))
    print("  expected output: {} ({:s})".format(target_idx, repr(idx2char[target_idx])))

Step    0
  input: 34 ('H')
  expected output: 55 ('a')
Step    1
  input: 55 ('a')
  expected output: 72 ('r')
Step    2
  input: 72 ('r')
  expected output: 72 ('r')
Step    3
  input: 72 ('r')
  expected output: 79 ('y')
Step    4
  input: 79 ('y')
  expected output: 1 (' ')


In [15]:
# Batch size 
BATCH_SIZE = 64
steps_per_epoch = examples_per_epoch//BATCH_SIZE

# Buffer size to shuffle the dataset
# (TF data is designed to work with possibly infinite sequences, 
# so it doesn't attempt to shuffle the entire sequence in memory. Instead, 
# it maintains a buffer in which it shuffles elements).
BUFFER_SIZE = 10000

dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)

dataset

<DatasetV1Adapter shapes: ((64, 100), (64, 100)), types: (tf.int32, tf.int32)>

In [16]:
# Length of the vocabulary in chars
vocab_size = len(vocab)

# The embedding dimension 
embedding_dim = 256

# Number of RNN units
rnn_units = 1024

In [17]:
if tf.test.is_gpu_available():
  rnn = tf.keras.layers.CuDNNGRU
else:
  import functools
  rnn = functools.partial(
    tf.keras.layers.GRU, recurrent_activation='sigmoid')

In [18]:
def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
  model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim, 
                              batch_input_shape=[batch_size, None]),
    rnn(rnn_units,
        return_sequences=True, 
        recurrent_initializer='glorot_uniform',
        stateful=True),
    tf.keras.layers.Dense(vocab_size)
  ])
  return model

In [19]:
model = build_model(
  vocab_size = len(vocab), 
  embedding_dim=embedding_dim, 
  rnn_units=rnn_units, 
  batch_size=BATCH_SIZE)

In [20]:
for input_example_batch, target_example_batch in dataset.take(1): 
  example_batch_predictions = model(input_example_batch)
  print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")

(64, 100, 325) # (batch_size, sequence_length, vocab_size)


In [21]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (64, None, 256)           83200     
_________________________________________________________________
cu_dnngru (CuDNNGRU)         (64, None, 1024)          3938304   
_________________________________________________________________
dense (Dense)                (64, None, 325)           333125    
Total params: 4,354,629
Trainable params: 4,354,629
Non-trainable params: 0
_________________________________________________________________


In [22]:
sampled_indices = tf.random.categorical(example_batch_predictions[0], num_samples=1)
sampled_indices = tf.squeeze(sampled_indices,axis=-1).numpy()
sampled_indices

array([300, 285, 110, 238, 102, 228, 152, 301, 270, 266,  12, 281, 127,
       168, 282, 122, 252, 317, 174, 218, 238, 282,  32,  45, 158,   1,
        33, 219, 184, 135, 184,  20, 193, 270, 124,  49,  47, 237, 206,
       182, 280, 135, 251, 321, 268, 259, 152, 195, 297, 143, 311, 225,
       249, 230, 216, 300, 235, 248, 292,  30, 252,  43, 188, 114,   1,
        21, 172, 206,  49, 165, 312, 131, 226,  54,  99,  17, 217, 272,
       259,  39,  33, 221, 183, 281, 292, 275,  78, 238, 165,  92, 181,
        98,  96, 298,  86,   0, 207,  70, 180, 238], dtype=int64)

In [23]:
print("Input: \n", repr("".join(idx2char[input_example_batch[0]])))
print()
print("Next Char Predictions: \n", repr("".join(idx2char[sampled_indices ])))

Input: 
 ' one I left behind\nThe Abduction\nDie Henkerstochter und der König der Bettler (Band 3 aus der Reihe '

Next Char Predictions: 
 '美流Пャūハغ者巡容.換р、撃л京黒さダャ撃FSن Gッのяの6り巡нWUムクな探я交엄屈図غを紙د錬ニ世パソ美マー的D京Qまв 7ぎクW’零цヌ]ā3タ師図MGデね換的彼xャ’ôとþü紳æ\nグpつャ'


In [24]:
def loss(labels, logits):
  return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)

example_batch_loss  = loss(target_example_batch, example_batch_predictions)
print("Prediction shape: ", example_batch_predictions.shape, " # (batch_size, sequence_length, vocab_size)") 
print("scalar_loss:      ", example_batch_loss.numpy().mean())

Prediction shape:  (64, 100, 325)  # (batch_size, sequence_length, vocab_size)
scalar_loss:       5.7831593


In [25]:
model.compile(
    optimizer = tf.train.AdamOptimizer(),
    loss = loss)

In [26]:
# Directory where the checkpoints will be saved
checkpoint_dir = './training_checkpoints'
# Name of the checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback=tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)

In [27]:
EPOCHS=50

In [28]:
history = model.fit(dataset.repeat(), epochs=EPOCHS, steps_per_epoch=steps_per_epoch, callbacks=[checkpoint_callback])

Epoch 1/50
 9/10 [==========================>...] - ETA: 0s - loss: 5.1475WARNING:tensorflow:From c:\users\red hot chili pooper\appdata\local\programs\python\python37\lib\site-packages\tensorflow\python\keras\engine\network.py:1436: update_checkpoint_state (from tensorflow.python.training.checkpoint_management) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.train.CheckpointManager to manage checkpoints rather than manually editing the Checkpoint proto.
10/10 [==============================] - 2s 224ms/step - loss: 5.0599
Epoch 2/50
10/10 [==============================] - 1s 139ms/step - loss: 3.7076
Epoch 3/50
10/10 [==============================] - 1s 137ms/step - loss: 3.4059
Epoch 4/50
10/10 [==============================] - 1s 141ms/step - loss: 3.2232
Epoch 5/50
10/10 [==============================] - 1s 142ms/step - loss: 2.9890
Epoch 6/50
10/10 [==============================] - 1s 137ms/step - loss: 2.7930
Epoch 7/50
10/10 [========

In [29]:
tf.train.latest_checkpoint(checkpoint_dir)

'./training_checkpoints\\ckpt_50'

In [30]:
model = build_model(vocab_size, embedding_dim, rnn_units, batch_size=1)

model.load_weights(tf.train.latest_checkpoint(checkpoint_dir))

model.build(tf.TensorShape([1, None]))

In [31]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (1, None, 256)            83200     
_________________________________________________________________
cu_dnngru_1 (CuDNNGRU)       (1, None, 1024)           3938304   
_________________________________________________________________
dense_1 (Dense)              (1, None, 325)            333125    
Total params: 4,354,629
Trainable params: 4,354,629
Non-trainable params: 0
_________________________________________________________________


In [32]:
def generate_text(model, start_string):
  # Evaluation step (generating text using the learned model)

  # Number of characters to generate
  num_generate = 1000

  # Converting our start string to numbers (vectorizing) 
  input_eval = [char2idx[s] for s in start_string]
  input_eval = tf.expand_dims(input_eval, 0)

  # Empty string to store our results
  text_generated = []

  # Low temperatures results in more predictable text.
  # Higher temperatures results in more surprising text.
  # Experiment to find the best setting.
  temperature = 0.65

  # Here batch size == 1
  model.reset_states()
  for i in range(num_generate):
      predictions = model(input_eval)
      # remove the batch dimension
      predictions = tf.squeeze(predictions, 0)

      # using a multinomial distribution to predict the word returned by the model
      predictions = predictions / temperature
      predicted_id = tf.multinomial(predictions, num_samples=1)[-1,0].numpy()
      
      # We pass the predicted word as the next input to the model
      # along with the previous hidden state
      input_eval = tf.expand_dims([predicted_id], 0)
      
      text_generated.append(idx2char[predicted_id])

  return (start_string + ''.join(text_generated))

In [33]:
print(generate_text(model, start_string='\n'))

Instructions for updating:
Use tf.random.categorical instead.

The Last Man Vey
The Negutions of the Dragon (The Mast Man, #2)
Missuredy
Die Henkerst (Cirque du Freak, #1)
The First Secret (The Wake of the Cold
Sughteron's Mission
Living Poirty
On the Witch of the Edge
The Magic Story
Magic Skepet
Clockwork Watch
Ammast Allut
The Bone Secret of the Castain of Nogary
The Regument Six
The Cross Man
The Mourt Latter Benie
Look Back of the Dead
The Dead Spriet
The Peepera Blake
The Dark of Twilogy
The Scarlet Prey
The Last Dang of The Dead Bitter
The Complete Remomist
The Woodsh of the Cone
The Bad Beg of the Dog
The Strange Callia Gondy
Shadowl's Cross
Secrets of the Sands
Batman: The Dead and the Menculum's The Derox of the Killing
Prince, #1)
The Birther Red
Pator the Sear
Magic Stails
The Nemer of Girls
The Killing Fare
The Paster of the Dragon
The Appoint of Stranger
The Colofers
Caril's Lavely The Charl Stepless
The Terring
Storm Freed
Book Mooner
The Markent Death
The Face of Death 